### Requirements: simplified_wiki and populated_samples generated by Data_Preprocessing.ipynb file

In [255]:
## choose whether to generate large or small dataset

## for custom dataset plaese give a different name and change the data split manually in the cells below

file = "small"

In [252]:
import pandas as pd
import numpy as np
import itertools
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as accuracy, f1_score as f1
from sklearn.metrics import precision_score as precision, recall_score as recall
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import linear_kernel

In [164]:
def label2int(label):
    if label == "SUPPORTS":
        return 0
    elif label == "REFUTES":
        return 1
    else:
        return 2
    
def int2label(num):
    if num == 0:
        return "SUPPORTS"
    elif num == 1:
        return "REFUTES"
    else:
        return "NOT ENOUGH INFO"

In [165]:
def save_sparse_csr(filename, array):
    # note that .npz extension is added automatically
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)

In [166]:
data = pd.read_json("simplified_wiki.jsonl", lines = True)
df = pd.read_json("populated_samples.jsonl", lines = True)

In [167]:
df

,label,claim,evidences
0,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[Nikolaj_Coster-Waldau, 7, He then played Det..."
1,SUPPORTS,Roman Atwood is a content creator.,"[[Roman_Atwood, 1, He is best known for his vl..."
2,SUPPORTS,"History of art includes architecture, dance, s...","[[History_of_art, 2, The subsequent expansion ..."
3,REFUTES,Adrienne Bailon is an accountant.,"[[Adrienne_Bailon, 0, Adrienne Eliza Houghton ..."
4,NOT ENOUGH INFO,System of a Down briefly disbanded in limbo.,[]
...,...,...,...
144725,REFUTES,Led Zeppelin released an eponymous debut album...,"[[Led_Zeppelin, 6, Although the group was init..."
144726,SUPPORTS,Taal was romantic.,"[[Taal_-LRB-film-RRB-, 0, Taal LRB English Rhy..."
144727,SUPPORTS,Her stars American actress Rooney Mara.,"[[Her_-LRB-film-RRB-, 3, The film also stars A..."
144728,SUPPORTS,J. R. R. Tolkien created Gimli.,"[[Gimli_-LRB-Middle-earth-RRB-, 0, Gimli is a ..."


In [168]:
data

,id,sentences,text
0,19_Kids_and_Counting,[19 Kids and Counting LRB formerly 17 Kids and...,19 Kids and Counting LRB formerly 17 Kids and ...
1,16th_IIFA_Awards,[The 2015 IIFA Awards officially known as the ...,The 2015 IIFA Awards officially known as the ...
2,12_Play,[12 Play is the debut studio album by American...,12 Play is the debut studio album by American ...
3,1998_NFL_season,[The 1998 NFL season was the 79th regular seas...,The 1998 NFL season was the 79th regular seaso...
4,12-hour_clock,[The 12 hour clock is a time convention in whi...,The 12hour clock is a time convention in which...
...,...,...,...
19903,Xochitlán_Todos_Santos,[Xochitl n Todos Santos LRB municipality RRB i...,Xochitln Todos Santos LRB municipality RRB is ...
19904,Øystein_Skar,[ystein Skar LRB born 15 October 1985 in Vinst...,ystein Skar LRB born 15 October 1985 in Vinstr...
19905,Överskottsbolaget,[oB LRB short for verskottsbolaget RRB is a Sw...,oB LRB short for verskottsbolaget RRB is a Swe...
19906,X_Marks_the_Spot_-LRB-1942_film-RRB-,[X Marks the Spot is a 1942 film noir crime fi...,X Marks the Spot is a 1942 film noir crime fil...


## Add code for split in case of custom Dataset split

In [253]:
train = 0
test = 0
if file == "small":
    t, test = train_test_split(df, test_size=0.05, random_state=1234, shuffle=True)
    p, train = train_test_split(t, test_size=0.10, random_state=1234, shuffle=True)
elif file == "large":
    train, test = train_test_split(df, test_size=0.15, random_state=1234, shuffle=True)
else:
    # your code here

In [210]:
train.reset_index(inplace = True)
test.reset_index(inplace = True)
print(test.label.value_counts())
print(train.label.value_counts())

SUPPORTS           12017
NOT ENOUGH INFO     5277
REFUTES             4416
Name: label, dtype: int64
SUPPORTS           67499
NOT ENOUGH INFO    30362
REFUTES            25159
Name: label, dtype: int64


In [211]:
train["label1"] = train.label.apply(label2int)
train.drop(["label", "index"], axis="columns", inplace=True)
train.rename(columns = {"label1": "label"}, inplace=True)

In [212]:
test["label1"] = test.label.apply(label2int)
test.drop(["label", "index"], axis="columns", inplace=True)
test.rename(columns = {"label1": "label"}, inplace=True)

In [250]:
test

,claim,evidences,label,docsID,eviID
0,Liv Tyler performed in The Leftovers.,"[[Liv_Tyler, 16, In 2014 Tyler made her televi...",0,[Liv_Tyler],[Liv_Tyler_16]
1,The Beatles played music in Hamburg.,[],2,[],[]
2,Reuters transmits news in Hindi.,[],2,[],[]
3,The French Revolution led to the Reign of Terr...,"[[French_Revolution, 21, The dictatorship impo...",1,"[Reign_of_Terror, French_Revolution]","[French_Revolution_21, Reign_of_Terror_0, Reig..."
4,Mrigayaa won at the 24th National Film Awards ...,[],2,[],[]
...,...,...,...,...,...
21705,Anne Bancroft won a Tony.,"[[Anne_Bancroft, 2, She won one Academy Award ...",0,[Anne_Bancroft],[Anne_Bancroft_2]
21706,Apollo 11 was NASA's Apollo program's fifth ma...,"[[Apollo_11, 8, Apollo 11 was launched by a Sa...",0,[Apollo_11],[Apollo_11_8]
21707,Kate Nash was dropped from her record label.,"[[Kate_Nash, 10, After being dropped by her re...",0,[Kate_Nash],[Kate_Nash_10]
21708,Gerard Lee was born in 1951.,"[[Gerard_Lee, 0, Gerard Lee LRB born 1951 in M...",0,[Gerard_Lee],[Gerard_Lee_0]


In [214]:
tf_idf = TfidfVectorizer()

In [215]:
vector = tf_idf.fit_transform(data.text)
train_vec = tf_idf.transform(train.claim.values.tolist())
test_vector = tf_idf.transform(test.claim.values.tolist())

In [216]:
len(tf_idf.vocabulary_)

113874

In [217]:
vector.shape

(19908, 113874)

In [218]:
train_vec.shape

(123020, 113874)

In [233]:
def get_top(data, samples, top=5):
    similarity = linear_kernel(samples, data)
    return np.argsort(similarity)[:, -top:][::]

In [234]:
def getRows(train):
    num = 0
    for i, row in train.iterrows():
        if(row.label == 2):
            num += 3
        else:
            num += len(row.evidences)
    return num

train_rows = getRows(train)
test_rows = getRows(test)

In [235]:
def makeDataset(samples, sample_vec, dim):
    t_text = []
    text = []
    combined = []
    j = 0
    similarity = np.zeros(shape = (dim, 1))
    for i, row in samples.iterrows():
        evi = []
        ## synthetic evidences for NOT ENOUGH INFO
        if row.label == 2:
            top_3 = get_top(vector, sample_vec[i], 3)
            synthetic = data.iloc[top_3[0]].reset_index()
            for s in synthetic.sentences:
                slist = list(filter(None, s))
                l = int(len(slist)/2)
                evi.append(slist[l])
        else:
            for e in row.evidences:
                evi.append(e[2])
        
        vec = tf_idf.transform(evi)
        sim = linear_kernel(sample_vec[i], vec)[0]
        
        for s in sim:
            similarity[j] = s
            j += 1
            
        combined.append([row.claim, evi, row.label])
        for e in evi:
            text.append(row.claim + " " + e)
            t_text.append([row.claim, e, row.label])
    
    t_data = tf_idf.transform(text)    
    t_data = sparse.hstack((t_data, similarity), format="csr")

    return combined, t_data, t_text

In [236]:
combined_train, train_data, train_text = makeDataset(train, train_vec, train_rows)

In [237]:
combined_test, test_data, test_text = makeDataset(test, test_vector, test_rows)

In [238]:
save_sparse_csr(f"train_{file}", train_data)

In [239]:
save_sparse_csr(f"test_{file}", test_data)

In [240]:
train_df = pd.DataFrame(train_text, columns=["claim", "evidence", "label"])
test_df = pd.DataFrame(test_text, columns=["claim", "evidence", "label"])

In [241]:
out = train_df.to_json(orient='records', lines=True)
file = f"bert_train_{file}.jsonl"

with open(file, 'w') as f:
    f.write(out)

out = test_df.to_json(orient='records', lines=True)
file = f"bert_test_{file}.jsonl"

with open(file, 'w') as f:
    f.write(out)

In [242]:
combined_train_df = pd.DataFrame(combined_train, columns=["claim", "evidence", "label"])
combined_test_df = pd.DataFrame(combined_test, columns=["claim", "evidence", "label"])

In [243]:
out = train_df.to_json(orient='records', lines=True)
file = f"bert_combined_train_{file}.jsonl"

with open(file, 'w') as f:
    f.write(out)

out = test_df.to_json(orient='records', lines=True)
file = f"bert_combined_test_{file}.jsonl"

with open(file, 'w') as f:
    f.write(out)

In [244]:
test_label = test_df[["label"]]
train_label = train_df[["label"]]

train_label

,label
0,2
1,2
2,2
3,0
4,0
...,...
285515,2
285516,0
285517,2
285518,2


In [245]:
out = train_label.to_json(orient='records', lines=True)
file = f"train_label_{file}.jsonl"

with open(file, 'w') as f:
    f.write(out)

out = test_df.to_json(orient='records', lines=True)
file = f"test_label_{file}.jsonl"

with open(file, 'w') as f:
    f.write(out)

In [259]:
filename = f"bert_retrieved_test_{file}.jsonl"
df = pd.read_json(filename, lines=True)
df

,claim,evidence,label
0,Liv Tyler performed in The Leftovers.,The series stars an ensemble cast featuring Ju...,0
1,Liv Tyler performed in The Leftovers.,Tyler has the nickname Rose Capital of America,0
2,Liv Tyler performed in The Leftovers.,Tyler is the principal city of the Tyler Metro...,0
3,Liv Tyler performed in The Leftovers.,Liv Rundgren Tyler LRB born Liv Rundgren July ...,0
4,Liv Tyler performed in The Leftovers.,In 2014 Tyler made her television debut on the...,0
...,...,...,...
36180,All My Children's setting is in Pine Valley.,Lake Valley California,0
36181,All My Children's setting is in Pine Valley.,All My Children became the third soap opera to...,0
36182,All My Children's setting is in Pine Valley.,Lake Valley LRB Nevada RRB Lincoln and White P...,0
36183,All My Children's setting is in Pine Valley.,All My Children was the first new network dayt...,0


In [206]:
def getDocsID(sample):
    ret = set()
    for s in sample:
        ret.add(s[0])
    return list(ret)

def getEviID(sample):
    ret = []
    for s in sample:
        ret.append(s[0] + "_" + str(s[1]))
    return ret

In [222]:
test["docsID"] = test.evidences.apply(getDocsID)
test["eviID"] = test.evidences.apply(getEviID)

In [223]:
test

,claim,evidences,label,docsID,eviID
0,Liv Tyler performed in The Leftovers.,"[[Liv_Tyler, 16, In 2014 Tyler made her televi...",0,[Liv_Tyler],[Liv_Tyler_16]
1,The Beatles played music in Hamburg.,[],2,[],[]
2,Reuters transmits news in Hindi.,[],2,[],[]
3,The French Revolution led to the Reign of Terr...,"[[French_Revolution, 21, The dictatorship impo...",1,"[Reign_of_Terror, French_Revolution]","[French_Revolution_21, Reign_of_Terror_0, Reig..."
4,Mrigayaa won at the 24th National Film Awards ...,[],2,[],[]
...,...,...,...,...,...
21705,Anne Bancroft won a Tony.,"[[Anne_Bancroft, 2, She won one Academy Award ...",0,[Anne_Bancroft],[Anne_Bancroft_2]
21706,Apollo 11 was NASA's Apollo program's fifth ma...,"[[Apollo_11, 8, Apollo 11 was launched by a Sa...",0,[Apollo_11],[Apollo_11_8]
21707,Kate Nash was dropped from her record label.,"[[Kate_Nash, 10, After being dropped by her re...",0,[Kate_Nash],[Kate_Nash_10]
21708,Gerard Lee was born in 1951.,"[[Gerard_Lee, 0, Gerard Lee LRB born 1951 in M...",0,[Gerard_Lee],[Gerard_Lee_0]


In [224]:
def getRetrievalData(samples, sample_vec, dim):
        
    top = get_top(vector, sample_vec)
    text = []
    neg_true_label = []
    t_text = []
    similarity = np.zeros(shape = (dim, 1))
    sim_index = 0
    for i, row in samples.iterrows():
        docs = data.iloc[top[i]]
        sent = []
        sentid = []
        for j, d in docs.iterrows():
            for k, s in enumerate(d.sentences):
                sent.append(s)
                sentid.append(d.id + "_" + str(k))
        
        vec = tf_idf.transform(sent)
        top_evi = get_top(vec, test_vector[i])        
        evi = []
        if row.label != 2:
            ids = row.eviID
            for e in top_evi[0]:
                evi.append(sent[e])
                if sentid[e] not in ids:
                    neg_true_label.append(0)
                else:
                    neg_true_label.append(1)

        else:
            for ev in top_evi[0]:
                evi.append(sent[ev])
                neg_true_label.append(1)

        vec = tf_idf.transform(evi)
        sim = linear_kernel(sample_vec[i], vec)
        sim = sim[0]
        for s in sim:
            similarity[sim_index] = s
            sim_index += 1

        for e in evi:
            text.append(row.claim + " " + e)
            t_text.append([row.claim, e, row.label])

    t_data = tf_idf.transform(text)
    t_data = sparse.hstack((t_data, similarity), format="csr")
    
    return t_data, t_text, neg_true_label

In [225]:
t_data, t_text, neg_true_label = getRetrievalData(test, test_vector, 5 * len(test.index))

In [226]:
t_data

<108550x113875 sparse matrix of type '<class 'numpy.float64'>'
	with 2563495 stored elements in Compressed Sparse Row format>

In [227]:
test_retrieved = pd.DataFrame(t_text, columns=["claim", "evidence", "label"])
neg_label = pd.DataFrame(neg_true_label, columns=["neg_label"])

In [228]:
neg_label = pd.concat([neg_label, test_retrieved.label], axis=1)
neg_label

,neg_label,label
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
108545,0,1
108546,0,1
108547,0,1
108548,0,1


In [229]:
out = neg_label.to_json(orient='records', lines=True)
file = f"neg_label_{file}.jsonl"

with open(file, 'w') as f:
    f.write(out)

In [230]:
test_retrieved

,claim,evidence,label
0,Liv Tyler performed in The Leftovers.,The series stars an ensemble cast featuring Ju...,0
1,Liv Tyler performed in The Leftovers.,Tyler has the nickname Rose Capital of America,0
2,Liv Tyler performed in The Leftovers.,Tyler is the principal city of the Tyler Metro...,0
3,Liv Tyler performed in The Leftovers.,Liv Rundgren Tyler LRB born Liv Rundgren July ...,0
4,Liv Tyler performed in The Leftovers.,In 2014 Tyler made her television debut on the...,0
...,...,...,...
108545,Ecosse Films refused to produce Monarch of the...,The Monarch of the Glen television series is l...,1
108546,Ecosse Films refused to produce Monarch of the...,Canada has complete sovereignty as an independ...,1
108547,Ecosse Films refused to produce Monarch of the...,The first book in that series is called The Mo...,1
108548,Ecosse Films refused to produce Monarch of the...,She remained monarch and Head of State of Cana...,1


In [231]:
out = test_retrieved.to_json(orient='records', lines=True)
file = f"bert_retrieved_test_{file}.jsonl"

with open(file, 'w') as f:
    f.write(out)

In [232]:
save_sparse_csr(f"test_retrieved_{file}", t_data)